In [2]:
#!pip install datasets pillow huggingface_hub


In [3]:
from google.colab import drive
drive.mount('/content/drive')
path='drive/MyDrive/Colab Notebooks/'


Mounted at /content/drive


In [ ]:
"""
import zipfile
with zipfile.ZipFile(path+'test.zip', 'r') as zip_ref:
    zip_ref.extractall(path)
"""

### Reading Data From Drive

# Prediction - VIT

In [4]:
from datasets import load_dataset


dataset = load_dataset("imagefolder", data_dir=path+'test')['train']
dataset

Resolving data files:   0%|          | 0/2200 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['image'],
    num_rows: 2200
})

### Definding our labels

In [5]:
labels=['baked_potato',
 'baklava',
 'caesar_salad',
 'cheese_sandwich',
 'cheesecake',
 'chicken',
 'chicken_curry',
 'chocolate_cake',
 'donuts',
 'eggs',
 'falafel',
 'fish',
 'french_fries',
 'hamburger',
 'hot_dog',
 'ice_cream',
 'lasagna',
 'omelette',
 'pizza',
 'spaghetti',
 'steak',
 'sushi',]

#https://github.com/openai/CLIP/blob/main/data/prompts.md for food101
prompt=['a photo of '+label+', a type of' for label in labels]




### Prediction - openai/clip-vit-large-patch14

In [6]:
from transformers import pipeline
checkpoint = "openai/clip-vit-large-patch14"
detector = pipeline(model=checkpoint, task="zero-shot-image-classification",device=0)
def find_predict(predictions):
  return [item[0]['label'] for item in predictions]
predictions = detector(dataset['image'], candidate_labels=prompt)
y_pred=find_predict(predictions)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

In [7]:
def prompt_to_label(x):
  for i in range(len(prompt)):
    if x==prompt[i]:
      return labels[i]

### Prediction to DataFrame

In [10]:
import os
import pandas as pd
import numpy as np
Files_Name = os.listdir(path+'test')
Files_Name.sort()
q1_submission_vit=pd.DataFrame(np.array([Files_Name ,[prompt_to_label(item) for item in y_pred]]).T,columns=['name','predicted'])
q1_submission_vit

,name,predicted
0,001a051b-8e6.jpg,baklava
1,001bfca4-b75.jpg,cheesecake
2,001f9917-8d1.jpg,fish
3,0020bbed-a6b.jpg,lasagna
4,0028f1c3-64a.jpg,hot_dog
...,...,...
2195,ff4e666f-a5c.jpg,chicken
2196,ff59a672-526.jpg,chicken
2197,ff95b6ab-cae.jpg,french_fries
2198,ffae3d32-778.jpg,omelette


### Saving Prediction

In [11]:
q1_submission_vit.to_csv(path+'q1_submission.csv', index=False)